In [1]:
from joblib import load
import cv2
import numpy as np
import mediapipe as mp
import time

In [2]:
model = load('jumpjack.joblib')
mpdraw = mp.solutions.drawing_utils
mppose = mp.solutions.pose
pose = mppose.Pose()
cap = cv2.VideoCapture(0)
ptime = 0
mk = 10
sum = 0
count = "count 0"

In [3]:
while True:
    ret, img = cap.read()
    image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = pose.process(image)
    pose_landmarks = results.pose_landmarks
    ctime = time.time()
    fps = 1/(ctime-ptime)
    ptime = ctime
    if pose_landmarks:
        mpdraw.draw_landmarks(image, results.pose_landmarks, mppose.POSE_CONNECTIONS)
        pose_landmarks = [[lmk.x, lmk.y, lmk.z] for lmk in pose_landmarks.landmark]
        frame_height, frame_width = img.shape[:2]
        pose_landmarks *= np.array([frame_width, frame_height, frame_width])
        pose_landmarks = np.around(pose_landmarks, 5).flatten().astype(np.int32).tolist()
        pd = str(model.predict([pose_landmarks]))
        if pd == "['jump']":
            mk = mk-1
        if mk == 0:
            sum = sum+1
            count = "count"+str(sum)
            mk = 8
        cv2.putText(image, count, (370, 50), cv2.FONT_HERSHEY_PLAIN,
                    3, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(image, str(int(fps)), (50, 50), cv2.FONT_HERSHEY_PLAIN,
                    3, (255, 0, 0), 3, cv2.LINE_AA)
        cv2.putText(image, pd, (150, 50), cv2.FONT_HERSHEY_PLAIN,
                    3, (255, 0, 0), 3)
    image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
    cv2.imshow("show", image)
    if cv2.waitKey(1) & 0xFF == ord("q"):
        break
cap.release()
cv2.destroyAllWindows()
